In [207]:
import pandas as pd


In [208]:
data = pd.read_csv("test.csv", header=None)
data.head()

,0,1
0,04/11/2022 09:06:01.346,MSKW 33 RAKA CB Back Prot Trip
1,04/11/2022 09:11:31.918,NKDA 33 HIMA 2 CB Open
2,04/11/2022 09:34:41.939,OWFS 33 NYZA 1 CB Open
3,04/11/2022 09:36:19.609,HOMA STATION SUPPLY FAIL Fault
4,04/11/2022 09:45:38.424,OFEX 132 GTX14 CB Open


In [209]:
def clean_data(data):
    # rename columns
    data.columns = ['datetime', 'event']
    # filter data 
    data = data[(data.datetime != "time") | (data.event != "event") ]
    #data["event"] = data["event"].str.lower().str.replace(" ","_")
    data["event"] = data["event"].str.lower()

    # covert datetime column to datetime object. SHOULD BE DONE AFTER FILTERING. ELSE, ERRORS
    data.datetime = pd.to_datetime(data.datetime)
    data["date"] = data.datetime.dt.date
    data["month"] = data.datetime.dt.month_name()
    data['weekday'] = data.datetime.dt.day_name()
    data["time"] = data.datetime.dt.time   

    return data



In [210]:
classify_list = ["trip cct fault","station supply fail","dc power failure","cb lockout locked","cb backup trip",
"cb back prot trip","dc fault","diff comm fail fault"]
def classify_events(row):
    #print(row)  
    for s in classify_list:
        if s in str(row):
            return "critical"

    # mind the indentation. Else, the for loop might check only the first event in the "classify_list"
    else:
        return "normal"

In [211]:
data = clean_data(data)
data.head()

<ipython-input-209-f663a24a8e42>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["event"] = data["event"].str.lower()
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-209-f663a24a8e42>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

,datetime,event,date,month,weekday,time
0,2022-04-11 09:06:01.346,mskw 33 raka cb back prot trip,2022-04-11,April,Monday,09:06:01.346000
1,2022-04-11 09:11:31.918,nkda 33 hima 2 cb open,2022-04-11,April,Monday,09:11:31.918000
2,2022-04-11 09:34:41.939,owfs 33 nyza 1 cb open,2022-04-11,April,Monday,09:34:41.939000
3,2022-04-11 09:36:19.609,homa station supply fail fault,2022-04-11,April,Monday,09:36:19.609000
4,2022-04-11 09:45:38.424,ofex 132 gtx14 cb open,2022-04-11,April,Monday,09:45:38.424000


In [212]:

data['type'] = data['event'].apply(lambda row: classify_events(row))
data.head()

,datetime,event,date,month,weekday,time,type
0,2022-04-11 09:06:01.346,mskw 33 raka cb back prot trip,2022-04-11,April,Monday,09:06:01.346000,critical
1,2022-04-11 09:11:31.918,nkda 33 hima 2 cb open,2022-04-11,April,Monday,09:11:31.918000,normal
2,2022-04-11 09:34:41.939,owfs 33 nyza 1 cb open,2022-04-11,April,Monday,09:34:41.939000,normal
3,2022-04-11 09:36:19.609,homa station supply fail fault,2022-04-11,April,Monday,09:36:19.609000,critical
4,2022-04-11 09:45:38.424,ofex 132 gtx14 cb open,2022-04-11,April,Monday,09:45:38.424000,normal


In [213]:
#critical_data.event.value_counts()

In [214]:
data["substation_acronym"] = data["event"].apply(lambda x: x.split()[0])
data["fault_voltage"] = data["event"].apply(lambda x: x.split()[1])
data.head()

,datetime,event,date,month,weekday,time,type,substation_acronym,fault_voltage
0,2022-04-11 09:06:01.346,mskw 33 raka cb back prot trip,2022-04-11,April,Monday,09:06:01.346000,critical,mskw,33
1,2022-04-11 09:11:31.918,nkda 33 hima 2 cb open,2022-04-11,April,Monday,09:11:31.918000,normal,nkda,33
2,2022-04-11 09:34:41.939,owfs 33 nyza 1 cb open,2022-04-11,April,Monday,09:34:41.939000,normal,owfs,33
3,2022-04-11 09:36:19.609,homa station supply fail fault,2022-04-11,April,Monday,09:36:19.609000,critical,homa,station
4,2022-04-11 09:45:38.424,ofex 132 gtx14 cb open,2022-04-11,April,Monday,09:45:38.424000,normal,ofex,132


In [215]:
critical_data = data[data.type == 'critical']
print(critical_data.shape)
critical_data.head()

(1648, 9)


,datetime,event,date,month,weekday,time,type,substation_acronym,fault_voltage
0,2022-04-11 09:06:01.346,mskw 33 raka cb back prot trip,2022-04-11,April,Monday,09:06:01.346000,critical,mskw,33
3,2022-04-11 09:36:19.609,homa station supply fail fault,2022-04-11,April,Monday,09:36:19.609000,critical,homa,station
10,2022-04-11 09:53:01.701,owfs 132 bb trip cct fault,2022-04-11,April,Monday,09:53:01.701000,critical,owfs,132
11,2022-04-11 09:53:20.255,owfs 132 bb trip cct fault,2022-04-11,April,Monday,09:53:20.255000,critical,owfs,132
18,2022-04-11 10:00:31.400,krma ups1 cubicle dc power failure fault,2022-04-11,April,Monday,10:00:31.400000,critical,krma,ups1


In [216]:
# default: keep="first"
# substations = data.drop_duplicates(["substation_acronym"])
# substations.to_csv("substation_acronyms1.csv")

In [217]:
substations = data.drop_duplicates(subset=['substation_acronym'], keep='first')
substations['substation_acronym'].to_csv("substation_acronyms2.csv")

In [218]:
excel_data = pd.read_excel("substation details.xlsx",sheet_name="substations", index_col='no')
excel_data.reset_index(inplace=True)
excel_data.head()

,no,substation_acronym,substation name,voltage,voltage levels,installed capacity (MW),location,generators,critical level,Comments
0,1,mskw,masaka west,220/132/33,3.0,540.0,central,0.0,3.0,supplies masaka city and surroundings
1,2,nkda,nkenda,220/132/33,3.0,40.0,western,1.0,4.0,IPPs and Hima cement
2,3,owfs,owen falls,132/66/33,3.0,380.0,eastern,1.0,5.0,evacuates over 380 MW
3,4,homa,hoima,132/33,2.0,40.0,western,1.0,3.0,supplies hoima city and surroundings. Evacuate...
4,5,ofex,owen falls extension,132/33,2.0,NaN,eastern,1.0,5.0,200 MW dam connected here


In [219]:
#data.to_csv("processed_data.csv")

## Combine dataframe. include substation details

In [222]:
## read df
substation_data_columns = ['substation_acronym',"critical level","substation name"]
substation_data = pd.read_excel("substation details.xlsx", index_col="no",sheet_name=0)
final_data = data.merge(substation_data[substation_data_columns], how="inner", on="substation_acronym")
final_data.head()

,datetime,event,date,month,weekday,time,type,substation_acronym,fault_voltage,critical level,substation name
0,2022-04-11 09:06:01.346,mskw 33 raka cb back prot trip,2022-04-11,April,Monday,09:06:01.346000,critical,mskw,33,3.0,masaka west
1,2022-04-11 12:25:43.629,mskw 33 raka cb back prot trip,2022-04-11,April,Monday,12:25:43.629000,critical,mskw,33,3.0,masaka west
2,2022-04-11 13:49:28.982,mskw 33 raka cb back prot trip,2022-04-11,April,Monday,13:49:28.982000,critical,mskw,33,3.0,masaka west
3,2022-04-11 17:18:25.427,mskw 33 raka cb back prot trip,2022-04-11,April,Monday,17:18:25.427000,critical,mskw,33,3.0,masaka west
4,2022-05-11 01:22:25.157,mskw 33 raka cb back prot trip,2022-05-11,May,Wednesday,01:22:25.157000,critical,mskw,33,3.0,masaka west


In [223]:
final_data.to_csv("final_data.csv")